In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from numpy.random import seed
seed(0)
tf.random.set_seed(0)
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import imblearn
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Permute
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking
#from keras.utils import plot_model
import sklearn
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Activation
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import optimizers 
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import gamma
from sklearn import metrics
from scipy.stats import reciprocal
from scipy.stats import randint
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#Load peripheral preprocessed data
#rsp_df = joblib.load(open('/home/sowmya/QoE_D2/Dataset2/SpyderCodeDataset2/data/rsp_18pyhrvftr_d2.dat', 'rb'))
#rsp_data= np.array(rsp_df)
rsp_df = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/rsp_40features.dat', 'rb'))
rsp_data= rsp_df.drop(['RRV_SampEn'], axis =1)
np.count_nonzero(np.isinf(rsp_data))

0

In [3]:
#Load peripheral preprocessed data
ecg_HRV, ratings = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/ECG_18pyHRVfeatures_new.dat', 'rb'))
ecg_morpho, ratings = joblib.load(open('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/SpyderCodeDataset2/data/ECG_morphofeatures.dat', 'rb'))

# Place the pyHRV and Morphological ff features DataFrames side by side
horizontal_stack = pd.concat([ecg_HRV, ecg_morpho, rsp_data], axis=1)
#drop columns with all NAN values
ff_df = horizontal_stack.dropna(axis=1, how='all')
#ff_df = rsp_data
ff_data= np.array(ff_df)


In [4]:
#Check if input contains zero and infinity or a value too large
np.count_nonzero(np.isnan(ff_data))
np.count_nonzero(np.isinf(ff_data))

#Indices containing inf values and convert them to NaN
indsinf = np.where(np.isinf(ff_data))
ff_data[indsinf] = 'NaN'

#Replace NaN values with mean of the column
col_mean = np.nanmean(ff_data, axis=0)    
inds = np.where(np.isnan(ff_data))
ff_data[inds] = np.take(col_mean, inds[1])
np.count_nonzero(np.isnan(ff_data))
np.count_nonzero(np.isinf(ff_data))

#ff= pd.DataFrame(data= ff_data, columns= ff_df.columns)

0

In [5]:
#load ratings from excel sheet as pandas dataframe
load_ratings = pd.read_excel ('C:/Users/Sowmya/QoE/QoE_D2/Dataset2/Ratings_Dataset2.xlsx', sheet_name='EI_ALL')
ratings1 = load_ratings.drop(['subject'], axis=1) #delete subject from dataframe
ff_label = ratings1.values #convert dataframe to numpy array

def data_binarizer_two(ratings):
	"""binarizes the data below and above the threshold"""
	binarized = []
	for rating in ratings:
		if rating <= 6:
			binarized.append(0)
		else:
			binarized.append(1)
	return binarized


from collections import Counter
y_ia = np.array(data_binarizer_two([el[0] for el in ff_label]))
y_aq = np.array(data_binarizer_two([el[1] for el in ff_label]))
y_iv = np.array(data_binarizer_two([el[2] for el in ff_label]))
y_il = np.array(data_binarizer_two([el[3] for el in ff_label]))
y_oq = np.array(data_binarizer_two([el[4] for el in ff_label]))
y_sa = np.array(data_binarizer_two([el[5] for el in ff_label]))

counter = Counter(y_il)
print(counter)


Counter({0: 317, 1: 223})


In [6]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)

'''
#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))
'''

smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108
   macro avg       0.79      0.81      0.80       108
weighted avg       0.82      0.81      0.82       108

KNN Accuracy: 0.814815
KNN F1score: 0.817043
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.77      0.75      0.76        36
           1       0.88      0.89      0.88        72

    accuracy                           0.84       108
   macro avg       0.82      0.82      0.82       108
weighted avg       0.84      0.84      0.84       108

RF Accuracy: 0.842593
RF F1score: 0.842027
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108
   macro avg       0.79      0.81      0.80       108
weighted avg       0.82      0.81      0.82       108

KNN Accuracy: 0.814815
KNN F1score: 0.817043
{'n_neighbors': 1}


In [7]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108
   macro avg       0.79      0.81      0.80       108
weighted avg       0.82      0.81      0.82       108

KNN Accuracy: 0.814815
KNN F1score: 0.817043
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.77      0.75      0.76        36
           1       0.88      0.89      0.88        72

    accuracy                           0.84       108
   macro avg       0.82      0.82      0.82       108
weighted avg       0.84      0.84      0.84       108

RF Accuracy: 0.842593
RF F1score: 0.842027
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4],

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.84      0.72      0.78        36
           1       0.87      0.93      0.90        72

    accuracy                           0.86       108
   macro avg       0.85      0.83      0.84       108
weighted avg       0.86      0.86      0.86       108

RF Random Accuracy _PARAMS: 0.861111
RF Random F1score _PARAMS: 0.858259
              precision    recall  f1-score   support

           0       0.79      0.75      0.77        36
           1       0.88      0.90      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.83      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Random Accuracy: 0.851852
RF Random F1score: 0.85075
              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.81      0.72      0.76        36
           1       0.87      0.92      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.82      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Accuracy: 0.851852
RF F1score: 0.849497
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
              precision    recall  f1-score   support

           0       0.79      0.72      0.75        36
           1       0.87      0.90      0.88        72

    accuracy                           0.84       108
   macro avg       0.83      0.81      0.82       108
weighted avg       0.84  

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [8]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108
   macro avg       0.79      0.81      0.80       108
weighted avg       0.82      0.81      0.82       108

KNN Accuracy: 0.814815
KNN F1score: 0.817043
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.84      0.75      0.79        36
           1       0.88      0.93      0.91        72

    accuracy                           0.87       108
   macro avg       0.86      0.84      0.85       108
weighted avg       0.87      0.87      0.87       108

RF Accuracy: 0.87037
RF F1score: 0.868309
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.79      0.75      0.77        36
           1       0.88      0.90      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.83      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Random Accuracy _PARAMS: 0.851852
RF Random F1score _PARAMS: 0.85075
              precision    recall  f1-score   support

           0       0.79      0.75      0.77        36
           1       0.88      0.90      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.83      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Random Accuracy: 0.851852
RF Random F1score: 0.85075
              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108


c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [9]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108
   macro avg       0.79      0.81      0.80       108
weighted avg       0.82      0.81      0.82       108

KNN Accuracy: 0.814815
KNN F1score: 0.817043
{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.81      0.69      0.75        36
           1       0.86      0.92      0.89        72

    accuracy                           0.84       108
   macro avg       0.83      0.81      0.82       108
weighted avg       0.84      0.84      0.84       108

RF Accuracy: 0.842593
RF F1score: 0.83936
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
# Split the data into training/testing sets
#X_train_ff_ia, X_test_ff_ia, y_train_ff_ia, y_test_ff_ia = train_test_split(ff_data, y_ia, test_size=0.2, random_state=42, stratify = y_ia)
X_train_ff_aq, X_test_ff_aq, y_train_ff_aq, y_test_ff_aq = train_test_split(ff_data, y_aq, test_size=0.2, random_state=42, stratify = y_aq)
#X_train_ff_iv, X_test_ff_iv, y_train_ff_iv, y_test_ff_iv = train_test_split(ff_data, y_iv, test_size=0.2, random_state=42, stratify = y_iv)
#X_train_ff_il, X_test_ff_il, y_train_ff_il, y_test_ff_il = train_test_split(ff_data, y_il, test_size=0.2, random_state=42, stratify = y_il)
#X_train_ff_oq, X_test_ff_oq, y_train_ff_oq, y_test_ff_oq = train_test_split(ff_data, y_oq, test_size=0.2, random_state=42, stratify = y_oq)
#X_train_ff_sa, X_test_ff_sa, y_train_ff_sa, y_test_ff_sa = train_test_split(ff_data, y_sa, test_size=0.2, random_state=42, stratify = y_sa)
  
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_ff_aq= sc.fit_transform(X_train_ff_aq)
X_test_ff_aq = sc.transform(X_test_ff_aq)


#principle component analysis
from sklearn.decomposition import PCA
pca = PCA(n_components=40)
X_train_ff_oq = pca.fit_transform(X_train_ff_aq)
X_test_ff_oq = pca.transform(X_test_ff_aq)

# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = ff_df.columns
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# using LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(sorted(dic.items()))


smote = SMOTE(random_state=0)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_ff_aq, y_train_ff_aq)
X_test_resampled, y_test_resampled = X_test_ff_aq, y_test_ff_aq

from collections import Counter
# summarize observations by class labeL
counter = Counter(y_train_resampled)
print(counter)

ff_randsearch = SVC(gamma= 'auto', random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))


from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneOut
rskfcv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
loocv = LeaveOneOut()


#from sklearn.utils.fixes import loguniform
ff_svm_param_grid= {'C': expon(scale=100), 'gamma': expon(scale=.1), 'kernel': ['rbf'], 'class_weight':['balanced', None]}
#ff_svm_param_grid= {'C': reciprocal(1e0, 1e3), 'gamma': reciprocal(1e-3, 1e-2), 'kernel': ['rbf'], 'class_weight':['balanced', None]}

ff_randsearch = RandomizedSearchCV(SVC(decision_function_shape='ovo'),param_distributions = ff_svm_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
#ff_randsearch.fit(X_train_ff_il, y_train_ff_il) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("SVM RandomCV Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("SVM RandomCV F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred,average = 'weighted'), 6))
print(ff_randsearch.best_params_)

#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
clf.fit(X_train_resampled, y_train_resampled) 
clf_pred = clf.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, clf_pred))
print("KNN Accuracy grid search:", round(metrics.accuracy_score(y_test_resampled, clf_pred), 6))
print("KNN F1score grid search:", round(metrics.f1_score(y_test_resampled, clf_pred, average = 'weighted'), 6))
print(clf.best_params_)

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)

ff_randsearch = RandomForestClassifier() 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

ff_randsearchRF = RandomizedSearchCV(RandomForestClassifier(),param_distributions = random_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0, verbose =2) 
ff_randsearchRF.fit(X_train_resampled, y_train_resampled) 
ff_randsearchRF_pred = ff_randsearchRF.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearchRF_pred))
print("RF Random Accuracy _PARAMS:", round(metrics.accuracy_score(y_test_resampled, ff_randsearchRF_pred), 6))
print("RF Random F1score _PARAMS:", round(metrics.f1_score(y_test_resampled, ff_randsearchRF_pred, average = 'weighted'), 6))


ff_randsearch = RandomForestClassifier(max_depth=10, random_state=0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("RF Random Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("RF Random F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))

ff_knn_param_grid= {'n_neighbors': (1,2,3,10, 20,100)}
ff_randsearch = RandomizedSearchCV(KNeighborsClassifier(),param_distributions = ff_knn_param_grid, n_iter = 100, n_jobs = -1, cv = 10, random_state = 0) 
ff_randsearch.fit(X_train_resampled, y_train_resampled) 
ff_randsearch_pred = ff_randsearch.predict(X_test_resampled)
print(metrics.classification_report(y_test_resampled, ff_randsearch_pred))
print("KNN Accuracy:", round(metrics.accuracy_score(y_test_resampled, ff_randsearch_pred), 6))
print("KNN F1score:", round(metrics.f1_score(y_test_resampled, ff_randsearch_pred, average = 'weighted'), 6))
print(ff_randsearch.best_params_)




Counter({0: 288, 1: 288})
              precision    recall  f1-score   support

           0       0.78      0.69      0.74        36
           1       0.86      0.90      0.88        72

    accuracy                           0.83       108
   macro avg       0.82      0.80      0.81       108
weighted avg       0.83      0.83      0.83       108

SVM Accuracy: 0.833333
SVM F1score: 0.830684
              precision    recall  f1-score   support

           0       0.92      0.31      0.46        36
           1       0.74      0.99      0.85        72

    accuracy                           0.76       108
   macro avg       0.83      0.65      0.65       108
weighted avg       0.80      0.76      0.72       108

SVM RandomCV Accuracy: 0.759259
SVM RandomCV F1score: 0.71627
{'C': 195.1545320209259, 'class_weight': None, 'gamma': 0.05510484910954993, 'kernel': 'rbf'}
              precision    recall  f1-score   support

           0       0.66      0.81      0.73        36
          

c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.84      0.75      0.79        36
           1       0.88      0.93      0.91        72

    accuracy                           0.87       108
   macro avg       0.86      0.84      0.85       108
weighted avg       0.87      0.87      0.87       108

RF Accuracy: 0.87037
RF F1score: 0.868309
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.79      0.75      0.77        36
           1       0.88      0.90      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.83      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Random Accuracy _PARAMS: 0.851852
RF Random F1score _PARAMS: 0.85075
              precision    recall  f1-score   support

           0       0.79      0.75      0.77        36
           1       0.88      0.90      0.89        72

    accuracy                           0.85       108
   macro avg       0.84      0.83      0.83       108
weighted avg       0.85      0.85      0.85       108

RF Random Accuracy: 0.851852
RF Random F1score: 0.85075
              precision    recall  f1-score   support

           0       0.70      0.78      0.74        36
           1       0.88      0.83      0.86        72

    accuracy                           0.81       108


c:\Users\aquar\anaconda3\envs\TF2\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
